In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

%load_ext autoreload
%autoreload 2

http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html
PCA on top?

In [ ]:
from crawler import async_cache_pages, urls_list
import random

In [ ]:
folders = ["general2"]

positives, negatives = urls_list(folders)
async_cache_pages(positives + negatives)

Error while downloading: https://business.udemy.com/?ref=ufb_header. Status code: 403


In [ ]:
# Same number of negative and positive examples
urls = positives + random.sample(negatives, len(positives))
labels = [True] * len(positives) + [False] * len(positives)

In [ ]:
from features import construct_text_df

In [ ]:
df = construct_text_df(urls, labels)

In [ ]:
df.sample(5, random_state=1)

In [ ]:
X = df["visible_text"]
y = df.label#.replace([False, True], [0, 1])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
count_vect = CountVectorizer()
tfidf_transformer = TfidfTransformer()

In [ ]:
X_counts = count_vect.fit_transform(X)

In [ ]:
X_tfidf = tfidf_transformer.fit_transform(X_counts)

In [ ]:
def top_tfidf_feats(row, features, top_n=25):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

def top_feats_in_doc(Xtr, features, row_id, top_n=25):
    ''' Top tfidf features in specific document (matrix row) '''
    row = np.squeeze(Xtr[row_id].toarray())
    return top_tfidf_feats(row, features, top_n)

In [ ]:
feats = count_vect.get_feature_names()
feats[10000:10000 + 10]

In [ ]:
top_feats_in_doc(X_tfidf, feats, 0, 10)

In [ ]:
top_feats_in_doc(X_tfidf, feats, 1, 10)

In [ ]:
top_feats_in_doc(X_tfidf, feats, 2, 10)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [ ]:
pipeline = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
])

pipeline.fit(X, y)

In [ ]:
docs_new = ["course submit", "education learning", "python coursera", "submit button", "medical hospital"]
predicted = pipeline.predict(docs_new)

In [ ]:
for doc, category in zip(docs_new, predicted):
    print(doc, "==>", category)

In [ ]:
from features import analyse_classification

In [ ]:
pipeline2 = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
])

analyse_classification(X, y, pipeline2)

## Taking all training data (more negatives than positives)

precision    recall  f1-score   support

      False       0.78      0.98      0.87        50
       True       0.92      0.44      0.59        25

avg / total       0.82      0.80      0.78        75

[[49  1]

 [14 11]]